In [3]:
!pip install torch

   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 1.0/212.5 MB 21.6 MB/s eta 0:00:10
    --------------------------------------- 3.1/212.5 MB 33.2 MB/s eta 0:00:07
   - -------------------------------------- 7.1/212.5 MB 50.3 MB/s eta 0:00:05
   -- ------------------------------------- 11.4/212.5 MB 73.1 MB/s eta 0:00:03
   -- ------------------------------------- 11.5/212.5 MB 73.1 MB/s eta 0:00:03
   -- ------------------------------------- 15.9/212.5 MB 65.6 MB/s eta 0:00:03
   --- ------------------------------------ 20.7/212.5 MB 73.1 MB/s eta 0:00:03
   --- ------------------------------------ 21.2/212.5 MB 54.7 MB/s eta 0:00:04
   ---- ----------------------------------- 25.5/212.5 MB 73.1 MB/s eta 0:00:03
   ----- ---------------------------------- 30.8/212.5 MB 72.6 MB/s eta 0:00:03
   ------ --------------------------------- 34.2/212.5 MB 93.9 MB/s eta 0:00:02
   ------- -------------------------------- 39.5/212

In [49]:
import sys
import melee
import random
import csv
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [52]:
# REPLACE WITH YOUR PATH NAME
pathToSlp = "replays\Game_20250422T225246.slp"
pathToReplays = "replays"

In [ ]:
console = melee.Console(system="file", path=pathToSlp)
console.connect()

while True:
    gamestate = console.step()
    # step() returns None when the file ends
    if gamestate is None:
        break
    for i, player in gamestate.players.items():
        print("player", i, "stock", player.stock)
        print("player", i,"percent", player.percent)
        print("player", i,"position x", player.position.x)
        print("player", i,"position y", player.position.y)
        print("player", i,"character", player.character)
        print("player", i, "action", player.action)
        print("player", i, "action frame", player.action_frame)
        print("player", i, "facing direction", player.facing)
        print("player", i, "shield strength", player.shield_strength)
        print("player", i, "jumps left", player.jumps_left)
        print("player", i, "is on ground", player.on_ground)
        print("player", i, "is invulnerable", player.invulnerable)
        print("player", i, "velocity air x self", player.speed_air_x_self)
        print("player", i, "velocity ground x self", player.speed_ground_x_self)
        print("player", i, "velocity x attack", player.speed_x_attack)
        print("player", i, "velocity y attack", player.speed_y_attack)
        print("player", i, "velocity y self", player.speed_y_self)
        break
    print("stage", gamestate.stage)
    for projectile in gamestate.projectiles:
        print("projectile type", projectile.type)
        print("projectile frame", projectile.frame)
        print("projectile owner", projectile.owner)
        print("projectile position x", projectile.position.x)
        print("projectile position y", projectile.position.y)
        print("projectile velocity x", projectile.speed.x)
        print("projectile velocity y", projectile.speed.y)
        break
    break

projectile type ProjectileType.UNKNOWN_PROJECTILE
projectile frame 1400
projectile owner 1
projectile position x 44.892292
projectile position y 1e-04
projectile velocity x 0.0
projectile velocity y 0.0
projectile type ProjectileType.UNKNOWN_PROJECTILE
projectile frame 1400
projectile owner 1
projectile position x 44.892292
projectile position y 1e-04
projectile velocity x 0.0
projectile velocity y 0.0
projectile type ProjectileType.UNKNOWN_PROJECTILE
projectile frame 1400
projectile owner 1
projectile position x 44.892292
projectile position y 1e-04
projectile velocity x 0.0
projectile velocity y 0.0
projectile type ProjectileType.UNKNOWN_PROJECTILE
projectile frame 1400
projectile owner 1
projectile position x 44.892292
projectile position y 1e-04
projectile velocity x 0.0
projectile velocity y 0.0
projectile type ProjectileType.UNKNOWN_PROJECTILE
projectile frame 1400
projectile owner 1
projectile position x 44.892292
projectile position y 1e-04
projectile velocity x 0.0
projectile 

In [ ]:
class MeleeDataset(Dataset):
    def __init__(self, states, actions):
        self.states = torch.stack(states)
        self.actions = torch.stack(actions)

    def __len__(self):
        return len(self.states)

    def __getitem__(self, idx):
        return self.states[idx], self.actions[idx]
    
def make_obs(gamestate, max_projectiles=5):
    # 1) Player features extractor (17 floats each)
    def player_feats(p):
        return [
            float(p.stock),
            float(p.percent),
            p.position.x,
            p.position.y,
            float(p.character.value),
            float(p.action.value),
            float(p.action_frame),
            float(p.facing),
            float(p.shield_strength),
            float(p.jumps_left),
            float(p.on_ground),
            float(p.invulnerable),
            p.speed_air_x_self,
            p.speed_ground_x_self,
            p.speed_x_attack,
            p.speed_y_attack,
            p.speed_y_self,
        ]

    # get player slots 1 & 2
    ps = gamestate.players
    f1 = player_feats(ps.get(1)) if 1 in ps else [0.0]*17
    #f2 = player_feats(ps.get(2)) if 2 in ps else [0.0]*17 #we dont want player2 data right?
    f2 = [float(gamestate.players.get(2).character.value)]

    # 2) Stage as one float
    stage_feat = [float(gamestate.stage.value)]

    # 3) Projectile features: each is 7 floats
    proj_feats = []
    for proj in gamestate.projectiles[:max_projectiles]:
        proj_feats.extend([
            float(proj.type.value),
            float(proj.frame),
            float(proj.owner),
            proj.position.x,
            proj.position.y,
            proj.speed.x,
            proj.speed.y,
        ])
    # pad out to max_projectiles * 7
    needed = max_projectiles*7 - len(proj_feats)
    if needed > 0:
        proj_feats.extend([0.0]*needed)

    all_feats = f1 + stage_feat + proj_feats f2
    return torch.tensor(all_feats, dtype=torch.float32)


In [56]:
def print_progress_bar(iteration, total, length=40):
    percent = iteration / total
    filled = int(length * percent)
    bar = '=' * filled + '-' * (length - filled)
    sys.stdout.write(f'\r[{iteration}/{total}] [{bar}] {percent:.1%}')
    sys.stdout.flush()

In [ ]:
slp_paths = glob.glob(os.path.join(pathToReplays, "*.slp"))
states, actions = [], []
total = len(slp_paths)


for idx, slp in enumerate(slp_paths, start=1):
    print_progress_bar(idx, total)
    console = melee.Console(system="file", path=slp)
    console.connect()
    while True:
        gs = console.step()
        if gs is None:
            break

        controller_state = gs.players[1].controller_state  # or whichever field holds the input
        act = torch.tensor([
            float(controller_state.button[melee.enums.Button.BUTTON_A]),
            float(controller_state.button[melee.enums.Button.BUTTON_B]),
            float(controller_state.button[melee.enums.Button.BUTTON_D_DOWN]),
            float(controller_state.button[melee.enums.Button.BUTTON_D_LEFT]),
            float(controller_state.button[melee.enums.Button.BUTTON_D_RIGHT]),
            float(controller_state.button[melee.enums.Button.BUTTON_D_UP]),
            float(controller_state.button[melee.enums.Button.BUTTON_L]),
            float(controller_state.button[melee.enums.Button.BUTTON_R]),
            float(controller_state.button[melee.enums.Button.BUTTON_X]),
            float(controller_state.button[melee.enums.Button.BUTTON_Y]),
            float(controller_state.button[melee.enums.Button.BUTTON_Z]),
            float(controller_state.button[melee.enums.Button.BUTTON_START]), # do we need this? @tony
            controller_state.main_stick[0], #x/y components
            controller_state.main_stick[1],
            controller_state.c_stick[0],
            controller_state.c_stick[1],
            controller_state.l_shoulder,
            controller_state.r_shoulder
        ], dtype=torch.float32)

        obs = make_obs(gs, max_projectiles=5)
        states.append(obs)
        actions.append(act)


dataset = MeleeDataset(states, actions)
loader  = DataLoader(dataset, batch_size=64, shuffle=True)


class PolicyNet(nn.Module):
    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_dim, 64), nn.ReLU(),
            nn.Linear(64, 64),      nn.ReLU(),
            nn.Linear(64, act_dim)
        )

    def forward(self, x):
        return self.net(x)

obs_dim = states[0].shape[0]
act_dim = actions[0].shape[0]
policy  = PolicyNet(obs_dim, act_dim)
opt     = optim.Adam(policy.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()  

# 5) TRAINING LOOP
for epoch in range(100):
    total_loss = 0.0
    for batch_states, batch_actions in loader:
        pred = policy(batch_states)
        loss = loss_fn(pred, batch_actions)
        opt.zero_grad()
        loss.backward()
        opt.step()
        total_loss += loss.item() * batch_states.size(0)
    avg_loss = total_loss / len(dataset)
    print(f"Epoch {epoch+1} — Loss: {avg_loss:.4f}")

[9/515] [----------------------------------------] 1.7%

In [ ]:
#pain